In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

from time import sleep

# widget stuff
import ipywidgets as widgets
from ipywidgets import interact, fixed

In [ ]:
# create geometry
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole
#DrawGeo(shape)

In [ ]:
# generate mesh out of geometry
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1)).Curve(3)
#Draw(mesh)

In [ ]:
# define finite element space for deformation function
fes_def = VectorH1(mesh,order=3,dirichlet="circle|top|bottom|inlet|outlet")

# trial and test functions, bilinear form
u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

# define grid functinos that later
# will hold the deformations
gf_def_top = GridFunction(fes_def) # upper boundary
gf_def_bot = GridFunction(fes_def) # lower boundary
gf_def_rad = GridFunction(fes_def) # radius of hole
gf_def = GridFunction(fes_def) # this will hold all deformations

# initial values for deformations
delta_top = Parameter(-0.3)
delta_bot = Parameter(0.3)
delta_rad = Parameter(0.2)

# update grid functions with actual deformations 
gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))
gf_def_bot.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("bottom"))
gf_def_rad.Set(5*CF((x-2.5, y-0.5)), definedon=mesh.Boundaries("circle"))

gf_def_top.vec.data -= A_def_inv@a_def.mat*gf_def_top.vec
gf_def_bot.vec.data -= A_def_inv@a_def.mat*gf_def_bot.vec
gf_def_rad.vec.data -= A_def_inv@a_def.mat*gf_def_rad.vec

# combine all deformations into one grid function
gf_def.vec.data = delta_top.Get() * gf_def_top.vec + \
                  delta_bot.Get() * gf_def_bot.vec + \
                  delta_rad.Get() * gf_def_rad.vec

Draw(gf_def)

In [ ]:
# define finite element space for deformation function
fes_def_2 = VectorH1(mesh,order=3,dirichlet="circle|top|bottom|inlet")

# trial and test functions, bilinear form
u_def_2, v_def_2 = fes_def_2.TnT()
a_def_2 = BilinearForm(InnerProduct(grad(u_def_2), grad(v_def_2))*dx).Assemble()
A_def_inv_2 = a_def_2.mat.Inverse(freedofs=fes_def_2.FreeDofs())

# define grid functinos that later
# will hold the deformations
gf_def_top_2 = GridFunction(fes_def_2) # upper boundary
gf_def_bot_2 = GridFunction(fes_def_2) # lower boundary
gf_def_2 = GridFunction(fes_def_2) # this will hold all deformations

# initial values for deformations
delta_top_2 = Parameter(0.5)
delta_bot_2 = Parameter(0.5)

# update grid functions with actual deformations 
gf_def_top_2.Set((0, exp(0.1*x)-1), definedon=mesh.Boundaries("top"))
gf_def_bot_2.Set((0, exp(0.1*x)-1), definedon=mesh.Boundaries("bottom"))

gf_def_top_2.vec.data -= A_def_inv_2@a_def_2.mat*gf_def_top_2.vec
gf_def_bot_2.vec.data -= A_def_inv_2@a_def_2.mat*gf_def_bot_2.vec

# combine all deformations into one grid function
gf_def_2.vec.data = delta_top_2.Get() * gf_def_top_2.vec + \
                    delta_bot_2.Get() * gf_def_bot_2.vec

Draw(gf_def_2) 
# 'deformation=gf_def_2' directly plots the deformation
# otherwise click 'Open Controls' > 'deformation'
Draw(gf_def_2, deformation=gf_def_2) 